In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [24]:
links = pd.read_csv('../data/links.csv', names = ['link'], skiprows=1)

In [25]:
links

,link
0,https://onebite.app/restaurant/pizza-hut-adana...
1,https://onebite.app/restaurant/del-sole-01-472...
2,https://onebite.app/restaurant/lava-lounge-piz...
3,https://onebite.app/restaurant/pizzaria-cosa-n...
4,https://onebite.app/restaurant/olivia-tjuvholm...
...,...
50582,https://onebite.app/restaurant/very-italian-pi...
50583,https://onebite.app/restaurant/donatellos-leid...
50584,https://onebite.app/restaurant/santa-lucia-lim...
50585,https://onebite.app/restaurant/pizze-pronto-sc...


In [142]:
a = links['link'][50586]

In [143]:
a

'https://onebite.app/restaurant/luigia-zurich-zh-851254c3'

In [29]:
r = requests.get(a)

In [34]:
bs = BeautifulSoup(r.text)

In [49]:
for s in bs.find_all('span'):
    print(s.string)

Home
None
Top Restaurants
None
Pizza Hut
None
Dave
0.0
None
 
Community
2.7
None
2 likes
Mon
1:00 PM - 10:00 PM
Tue
1:00 PM - 10:00 PM
Wed
1:00 PM - 10:00 PM
Thu
1:00 PM - 10:00 PM
Fri
1:00 PM - 10:00 PM
Sat
1:00 PM - 10:00 PM
Sun
1:00 PM - 10:00 PM
sammy.courtney
7/23/19, 9:41 a.m.
37 views
•
2 likes
bailey.redlinger
1/24/19, 9:10 a.m.
0 likes


In [148]:
for s in bs.find_all('a'):
    if '84601126' in (s.attrs['class']):
        print(s.text)

In [153]:
for s in bs.find_all('a'):
    if 'jsx-84601126' in (s.attrs['class']):
        print(s.text)

Pizza Hut
213. Sok., Adana, 01


In [135]:
for s in bs.find_all('span'):
    if 'score' in s.attrs['class']:
        print(s.next)

0.0
2.7


In [129]:
s.next_elements

<generator object PageElement.next_elements at 0x117fe5ed0>

In [126]:
s.next_element

'0 likes'

In [137]:
for s in bs.find_all('span'):
    print(s)

<span class="jsx-3358343937">Home</span>
<span class="jsx-3358343937 arrow"><svg style="width:9px;height:12px;transform:rotate(90deg)" viewbox="0 0 24 24"><defs></defs><g fill="none" fill-rule="evenodd" stroke="none" stroke-width="1"><mask fill="white" id="arrow-mask-2"><use href="#arrow-path-1"></use></mask><g transform="translate(2.000000, 6.000000)"><path d="M10,3.99427406 L1.99918201,11.9950921 L-2.73558953e-13,9.99591004 L9.99591004,-2.73780998e-13 L10,0.00408995583 L10.00409,-2.77111667e-13 L20,9.99591004 L18.000818,11.9950921 L10,3.99427406 Z" fill="#999999" id="arrow-path-1"></path></g></g></svg></span>
<span class="jsx-3358343937">Top Restaurants</span>
<span class="jsx-3358343937 arrow"><svg style="width:9px;height:12px;transform:rotate(90deg)" viewbox="0 0 24 24"><defs></defs><g fill="none" fill-rule="evenodd" stroke="none" stroke-width="1"><mask fill="white" id="arrow-mask-2"><use href="#arrow-path-1"></use></mask><g transform="translate(2.000000, 6.000000)"><path d="M10,3.